# NPS Corporate

In [1]:
#import pandas as pd
import re
from IPython.display import Markdown, HTML, display, FileLink
from ThaiTextPrepKit import polars_pretextkit as preprocess, typo_patterns as TYPO
import polars as pl
import pythainlp

KeyboardInterrupt: 

In [ ]:
def highlight_patterns(patterns, text, html=True, highlight_color_replace="#FFFF00", highlight_color_match="#00FFFF"):
    """
    Highlights all occurrences of the given regex patterns in the text using HTML <span> tags with background color.

    :param patterns: A list of tuples containing regex patterns and their replacements.
    :param text: The text to search within.
    :param highlight_color_replace: The background color to use for replacements (default is yellow).
    :param highlight_color_match: The background color to use for matches without replacement (default is blue).
    :return: The text with highlighted matches.
    """
    def add_highlight(match, color):
        return f'<span style="background-color: {color};">{match.group(0)}</span>' if html else f'<typo>{match.group(0)}</typo>'
    
    highlighted_text = text
    for pattern, replacement in patterns:
        matches = list(re.finditer(pattern, text))
        replacement = replacement.lstrip('<IGNORE>').rstrip('</IGNORE>')
        if matches:
            for match in matches:
                if replacement in match.group(0):
                    highlighted_text = highlighted_text.replace(match.group(0), add_highlight(match, highlight_color_match))
                else:
                    highlighted_text = highlighted_text.replace(match.group(0), add_highlight(match, highlight_color_replace))
    
    return highlighted_text

In [ ]:
patterns = TYPO.patterns

In [ ]:
def get_highlight_texts(patterns, texts: list) -> None:
    html_text = []
    for text in texts:
        html_text.append(highlight_patterns(patterns, text))
    return html_text

In [ ]:
text_cols = [
    'Text',
]

In [ ]:
path = r"C:\Users\patom\OneDrive\Documents\Repo\DataPrepKit\Samples\(QC) Social_4.xlsx"
#path = "/Users/pa/Documents/GitHub Repository Clone/DataPrepKit/Samples/NPS Reason HY'24-textprocessing test1.xlsx"

df = pl.read_excel(path,
                   engine='calamine')

concat_df = df.with_columns(
    pl.concat_list(text_cols).alias('text')
)

In [ ]:
concat_df = concat_df.explode('text')
concat_df = concat_df.drop_nulls('text')

In [ ]:
concat_df.shape

(14304, 46)

In [ ]:
original_texts = concat_df.get_column('text').to_list()

In [ ]:
html_texts = get_highlight_texts(patterns, original_texts)

for text in html_texts[:5]:
    display(Markdown(text))

ที่<span style="background-color: #FFFF00;">แอพ</span>ของ AEON จะมีการให้เรา<span style="background-color: #00FFFF;">เลือก</span>ครับว่าจะจ่ายขั้นต่ำหรือไม่

สมัคร บัตรเครดิต อิออน ผ่าน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอพ</span></span> รับบัตรสาขาได้ไหม สมัคร บัตรเครดิต อิออน ผ่าน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอพ</span></span>ขึ้นผ่าน<span style="background-color: #00FFFF;">แล้ว</span> รับบัตรสาขาได้ไหม

สมัครบัตร Aeon ใน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอป</span></span> ขึ้นอนุมัติ<span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;">แล้ว</span></span></span>จะได้บัตรภายในกี่วันครับ สมบัตรอีอ้อนผ่าน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอป</span></span> <span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;">แล้ว</span></span></span>ขึ้นว่าบัตรอนุมัติ<span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;">แล้ว</span></span></span> จะได้บัตรหรือเลขบัตรภายในกี่วันครับ

สมัคร บัตรเครดิต อิออน ผ่าน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอพ</span></span> สมัคร บัตรเครดิต อิออน ผ่าน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอพ</span></span> <span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;">แล้ว</span></span>บัตรไม่มาสักที มีคนโทรมาบอกว่ามาส่ง<span style="background-color: #00FFFF;"><span style="background-color: #00FFFF;">แล้ว</span></span>แต่ไม่เห้นมี สามารถเช้คได้ที่ไหน

สมัคร บัตรเครดิต อิออน ผ่าน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอพ</span></span> ขอเลข EMS หน่อยครับ 0954902940 สมัคร บัตรเครดิต อิออน ผ่าน<span style="background-color: #FFFF00;"><span style="background-color: #FFFF00;">แอพ</span></span> ขอเลข EMS หน่อยครับ 0954902940

In [ ]:
#skip_patterns = [re.compile(rf'(เบอ[ร]*[์]*โท[ร]*[สศ]ั[พบ][ท]*[์]*)|(เบอ[ร์]*โท[ร]*)|(เบอ(?!ะ)[ร]*[์]*)', re.IGNORECASE),]

In [ ]:
series = pl.Series(original_texts)
pre_series = preprocess.preprocess_text_batches(series=series,
                                   custom_dict=None,
                                   keep_stopwords=True,
                                   keep_format=True,
                                   return_token_list=False,
                                   lower_case=False,
                                   include_pattern='[/()]',
                                   skip_patterns=None,
                                   patterns=patterns)

for t in pre_series[:5]:
    print(t)

ที่แอปพลิเคชันของ AEON จะมีการให้เราเลือกครับว่าจะจ่ายขั้นต่ำหรือไม่
สมัคร บัตรเครดิต อิออน ผ่านแอปพลิเคชัน รับบัตรสาขาได้ไหม สมัคร บัตรเครดิต อิออน ผ่านแอปพลิเคชันขึ้นผ่านแล้ว รับบัตรสาขาได้ไหม
สมัครบัตร Aeon ในแอปพลิเคชัน ขึ้นอนุมัติแล้วจะได้บัตรภายในกี่วันครับ สมบัตรอีอ้อนผ่านแอปพลิเคชัน แล้วขึ้นว่าบัตรอนุมัติแล้ว จะได้บัตรหรือเลขบัตรภายในกี่วันครับ
สมัคร บัตรเครดิต อิออน ผ่านแอปพลิเคชัน สมัคร บัตรเครดิต อิออน ผ่านแอปพลิเคชัน แล้วบัตรไม่มาสักที มีคนโทรมาบอกว่ามาส่งแล้วแต่ไม่เห้นมี สามารถเช้คได้ที่ไหน
สมัคร บัตรเครดิต อิออน ผ่านแอปพลิเคชัน ขอเลข EMS หน่อยครับ 0954902940 สมัคร บัตรเครดิต อิออน ผ่านแอปพลิเคชัน ขอเลข EMS หน่อยครับ 0954902940


In [ ]:
def generate_html_table(*args):
    """
    Generate an HTML table with columns for each text list provided, using the given names as headers.

    Args:
    - *args (tuple): Variable number of tuples, each containing a list of texts and a corresponding name.

    Returns:
    - html_content (str): String containing the HTML table.
    """
    headers = [name for _, name in args]
    
    html_content = "<table border='1'>\n"
    html_content += "<tr>" + "".join(f"<th>{header}</th>" for header in headers) + "</tr>\n"
    
    # Find the maximum length among the provided lists to handle uneven lengths
    max_length = max(len(text_list) for text_list, _ in args)
    
    for i in range(max_length):
        html_content += "<tr>"
        for text_list, _ in args:
            cell_content = text_list[i] if i < len(text_list) else ""
            html_content += f"<td>{cell_content}</td>"
        html_content += "</tr>\n"

    html_content += "</table>"
    
    return html_content

In [ ]:
# Generate HTML table
html_table = generate_html_table((html_texts, 'raw text'), 
                                 (get_highlight_texts(patterns, pre_series.to_list()), "PAREPA"),
                                 #(pn_raw_token_texts, 'pn engine detection'),
                                 #(pn_pre_token_texts, 'pn engine correction')
                                 )

# Display HTML in Jupyter Notebook
from IPython.display import HTML, display
#display(HTML(html_table))

In [ ]:
# Create a downloadable link for the HTML content
with open('table_social_pre_text.html', 'w', encoding='utf-8') as f:
    f.write(html_table)

FileLink('table_social_pre_text.html')

c:\Users\patom\OneDrive\Documents\Repo\DataPrepKit\table_social_pre_text.html